# Classification
In the first cell all the english messages about cybersecurity groups of the GroupMonitoringRelease database will be classified and saved in the corresponding csv file.

In [ ]:
import os
import re
import torch
import emoji
import csv
import pandas as pd
from pymongo import MongoClient
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datetime import datetime
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
MONGO_URI = os.getenv("MONGO_URI")
DB_NAME = "GroupMonitoringRelease"
PERCORSO_MODELLO = "/kaggle/input/model-securebert/pytorch/default/1/modello_finale"
SOGLIA_CYBER = 0.5  # Alzata leggermente per ridurre falsi positivi (Replied, ecc.)

# Cartella Output
OUTPUT_FOLDER = "Results_CSV"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# LISTA COMPLETA GRUPPI CYBER INGLESI (Targettizzati)
GRUPPI_DA_ANALIZZARE = [
    # 1. Gruppi "Core" Cybersecurity & InfoSec
    "Cyber Security - Information Security - IT Security - Experts",
    "Hacking Realm",
    "VirusCheck Chat",  # Ottimo per hash di malware
    
    # 2. Network Security & Attacks (DDoS, Tunneling)
    "DDOS交流攻击",     # Attacchi DDoS (Spesso pieno di IP target)
    "HTTP Injector",    # Network Tunneling (Usato spesso per bypass)
    "HTTP Injector Chat",
    "Mikrotik-Training", # Configurazione Router/Firewall (Spesso discussioni su attacchi)
    
    # 3. Darknet & Underground (Russian/English Mixed)
    "Only Dark",
    "Теневой Даркнет Чат", # "Shadow Darknet Chat"
    "Хакеры |Чат| 𝓗𝓪𝓬𝓴𝓮𝓻𝓼 𝓬𝓱𝓪𝓽",
    
    # 4. Attori Malevoli Specifici
    "Чат доксеров",     # Doxing (Spesso condividono leak di dati personali)
    "HackDroids — Чат"  # Hacking Android / Mobile Malware
]

LIMITE_MESSAGGI_PER_GRUPPO = None # Analizza tutto lo storico

# ==============================================================================
# 2. CARICAMENTO MODELLO (GPU CHECK)
# ==============================================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Caricamento modello su: {device.upper()}...")

try:
    tokenizer = RobertaTokenizer.from_pretrained(PERCORSO_MODELLO)
    model = RobertaForSequenceClassification.from_pretrained(PERCORSO_MODELLO).to(device)
    model.eval()
    print("✅ Modello caricato e pronto.\n")
except Exception as e:
    print(f"❌ ERRORE: Impossibile caricare il modello. {e}")
    exit()

# ==============================================================================
# 3. FUNZIONI (Clean & Classify - NO TRANSLATION)
# ==============================================================================
def clean_and_mask(text):
    if not isinstance(text, str): return ""
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'CVE-\d{4}-\d+', '[CVE]', text, flags=re.IGNORECASE)
    text = re.sub(r'(?:https?://)?(?:www\.)?(?:t\.me|telegram\.me)/[a-zA-Z0-9_]+', '[TG_LINK]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '[IP]', text)
    text = re.sub(r'\b(?:[a-zA-Z0-9-]+\.)+(?:com|org|net|io|ru|cn|it|uk|gov)\b', '[DOMAIN]', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def classifica_messaggio_batch(testi):
    """Classifica una lista di messaggi in una volta sola (Batch Processing per GPU)"""
    testi_puliti = [clean_and_mask(t) for t in testi]
    
    try:
        # Tokenizzazione Batch
        inputs = tokenizer(testi_puliti, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        # Restituisce la colonna 1 (Probabilità Cyber) come lista Python
        return probs[:, 1].tolist()
    except Exception as e:
        # Fallback in caso di errore strano su un batch
        return [0.0] * len(testi)

# ==============================================================================
# 4. CONNESSIONE DB & CORE LOGIC
# ==============================================================================
print("🔌 Connessione a MongoDB...")
try:
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    db = client[DB_NAME]
    groups_coll = db["groups"]
    client.server_info() # Trigger connection check
except Exception as e:
    print(f"❌ Errore connessione DB: {e}")
    print("⚠️ Se sei su Kaggle, verifica che l'IP dell'Università accetti connessioni esterne!")
    exit()

query_gruppi = {"chat_name": {"$in": GRUPPI_DA_ANALIZZARE}}
target_groups = list(groups_coll.find(query_gruppi))
print(f"📋 Trovati {len(target_groups)} gruppi target.")

# ==============================================================================
# 5. CICLO DI ANALISI OTTIMIZZATO (BATCHING)
# ==============================================================================
BATCH_SIZE = 32 # Numero di messaggi analizzati contemporaneamente dalla GPU

for group_doc in target_groups:
    coll_name = group_doc.get("collection_name")
    chat_name = group_doc.get("chat_name", "Unknown")
    
    # Pulizia nome file (rimuove caratteri illegali per Windows/Linux)
    safe_filename = re.sub(r'[\\/*?:"<>|]', "", chat_name).replace(" ", "_")
    csv_path = os.path.join(OUTPUT_FOLDER, f"{safe_filename}.csv")
    
    if not coll_name: continue

    print(f"\n{'='*60}")
    print(f"📂 Gruppo: {chat_name}")
    
    msg_coll = db[coll_name]
    cursor = msg_coll.find({}).sort("date", -1)
    if LIMITE_MESSAGGI_PER_GRUPPO: cursor = cursor.limit(LIMITE_MESSAGGI_PER_GRUPPO)
    
    # Carichiamo in memoria per fare batching (se RAM < 16GB, ridurre query limit)
    msgs = list(cursor)
    total_msgs = len(msgs)
    print(f"   Messaggi totali: {total_msgs}")
    
    if total_msgs == 0: continue

    risultati_gruppo = []
    
    # Processiamo a blocchi (Batch)
    for i in tqdm(range(0, total_msgs, BATCH_SIZE), desc="GPU Processing"):
        batch_docs = msgs[i : i + BATCH_SIZE]
        
        # Filtriamo testi validi
        valid_texts = []
        valid_indices = []
        
        for idx, doc in enumerate(batch_docs):
            text = doc.get("message", "")
            if text and len(text) > 4:
                valid_texts.append(text)
                valid_indices.append(idx)
        
        if not valid_texts: continue
        
        # INFERENZA GPU
        scores = classifica_messaggio_batch(valid_texts)
        
        # Salvataggio risultati batch
        for local_idx, score in zip(valid_indices, scores):
            if score >= SOGLIA_CYBER:
                doc = batch_docs[local_idx]
                risultati_gruppo.append({
                    "message_id": doc.get("id"),
                    "date": doc.get("date"),
                    "user": doc.get("from_id"), # O username se disponibile
                    "text": valid_texts[valid_indices.index(local_idx)], # Testo originale
                    "score": round(score, 4)
                })

    # SALVATAGGIO CSV DEL GRUPPO
    if risultati_gruppo:
        df_res = pd.DataFrame(risultati_gruppo)
        df_res.to_csv(csv_path, index=False)
        print(f"   ✅ Salvato: {csv_path} ({len(risultati_gruppo)} minacce)")
    else:
        print("   💤 Nessuna minaccia trovata in questo gruppo.")

print("\n" + "="*60)
print(f"🏁 Finito! Controlla la cartella '{OUTPUT_FOLDER}'")
client.close()

2025-11-25 14:04:14.261520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764079454.459540      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764079454.514956      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

   ✅ Salvato: Results_CSV/Чат_доксеров.csv (120895 minacce)

📂 Gruppo: HackDroids — Чат
   Messaggi totali: 654


GPU Processing: 100%|██████████| 21/21 [00:04<00:00,  4.80it/s]


   ✅ Salvato: Results_CSV/HackDroids_—_Чат.csv (255 minacce)

📂 Gruppo: VirusCheck Chat
   Messaggi totali: 18505


GPU Processing: 100%|██████████| 579/579 [01:30<00:00,  6.43it/s]


   ✅ Salvato: Results_CSV/VirusCheck_Chat.csv (5838 minacce)

🏁 Finito! Controlla la cartella 'Results_CSV'
